In [ ]:
import pandas as pd
import simpy

In [ ]:
base_dados = pd.read_csv("DoaçãoRim.csv", sep=",")

display(base_dados)

## Mantendo apenas os dados que serão utilizados 

In [ ]:
remover = base_dados.drop(columns=["age_cat","time_on_Dialysis","prior_transplant","date_acutal","Time_death","Transplant_Y_N","X36MthsTx","Time_Tx","event","razon_removed"])

remover.head()

In [ ]:
base_dados = pd.DataFrame(base_dados['date'])
vetor = base_dados.value_counts
print(vetor)


In [ ]:



def fila(env, recurso, paciente, saida, chegada, prioridade):
    yield env.timeout(chegada)
    yield env.timeout(saida)
    if saida != 0:
        print(paciente, 'saiu da fila em', saida)
    else:
        with recurso.request(priority=prioridade) as req:
            yield req
            print(paciente, 'atendida em', env.now)
            yield env.timeout(5)
            print(paciente, 'saiu do atendimento em', env.now)

env = simpy.Environment()
recurso = simpy.PriorityResource(env, capacity=1)

env.process(fila(env, recurso, 'Maria', 0, 0, 0))
env.process(fila(env, recurso, 'Paula', 2, 2, 1))
env.process(fila(env, recurso, 'Paulo', 6, 4, 1))
env.process(fila(env, recurso, 'Alana', 0, 0, 1))
env.process(fila(env, recurso, 'Sandra', 0, 0, 0))

env.run()

